In [10]:
# Import necessary packages
import pandas as pd

# Load the data
input_csv_file = input('Enter the name of the csv file : ')
df = pd.read_csv(input_csv_file, encoding="ISO-8859-1")

In [2]:
df.shape
name = 'superstore_final_dataset'

In [3]:
def is_not_number(value):
    try:
        float_value = float(value)
        return not isinstance(float_value, (int, float))
    except ValueError:
        return True

In [11]:
# Randomly shuffle the rows in the dataframe 
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Number of lines for json
json_percent = (input('Enter the JSON percentage : '))
json_percent = 30 if is_not_number(json_percent) else float(json_percent)
line_json = int((shuffled_df.shape[0] * json_percent) / 100)

# Number of lines for csv
csv_percent = (input('Enter the CSV percentage : '))
csv_percent = 40 if is_not_number(csv_percent) else float(csv_percent)
line_csv = int((shuffled_df.shape[0] * csv_percent) / 100)

# Number of lines for database
db_percent = (input('Enter the database percentage : '))
db_percent = 30 if is_not_number(db_percent) else float(db_percent)
line_db = int((shuffled_df.shape[0] * db_percent) / 100)

### Json 

In [12]:
# Take 30% to json
json_df = shuffled_df[:line_json]

# Delete the json df from the shuffled
shuffled_df.drop(json_df.index, axis='rows', inplace=True)

# Create json file
json_df.reset_index(drop=True).to_json('json_file.json')

In [6]:
shuffled_df.shape

(6860, 18)

In [18]:
json_df.shape

(2940, 18)

### CSV

In [13]:
# 40% for CSV
csv_df = shuffled_df[:line_csv]

# Delete the csv df to the shuffle
shuffled_df.drop(csv_df.index, axis='rows', inplace=True)

# Create the csv file
csv_df.reset_index(drop=True).to_csv('csv_file.csv', index=False, header=False)

In [9]:
shuffled_df.shape

(0, 18)

In [10]:
csv_df.shape

(3920, 18)

### Database

In [14]:
# 30% for DB
db_df = shuffled_df[:line_db]

# Delete the csv df to the shuffle
shuffled_df.drop(db_df.index, axis='rows', inplace=True)

In [15]:
# Import SQL Alchemy
from sqlalchemy import create_engine, Column, Integer, String, Text, Boolean, Float, Date, text, inspect, UniqueConstraint
from sqlalchemy.orm import sessionmaker, declarative_base

# Construct the connection string
driver = 'ODBC+Driver+18+for+SQL+Server'
server_address = 'localhost'
database_name = 'Superstore' 
username = 'SA' 
password = 'YourPassword123'  

# Connection string for master db
master_connection_string = f'mssql+pyodbc://{username}:{password}@{server_address}/{database_name}?driver={driver}&Encrypt=No'

# Engine for master
master_engine = create_engine(master_connection_string)

In [16]:
# Create class name
def class_name(filename):
    filename = str.replace(filename, '.csv', '')
    filename = str.replace(filename, '_', ' ')
    filename = str.replace(filename, '-', ' ')

    filename = str.title(filename)
    filename = str.replace(filename, ' ', '')
    return filename

In [17]:
# Transform date type
db_df['Order_Date'] = pd.to_datetime(db_df['Order_Date'], dayfirst=True)
db_df['Ship_Date'] = pd.to_datetime(db_df['Ship_Date'], dayfirst=True)

/tmp/ipykernel_5178/666983151.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_df['Order_Date'] = pd.to_datetime(db_df['Order_Date'], dayfirst=True)
/tmp/ipykernel_5178/666983151.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_df['Ship_Date'] = pd.to_datetime(db_df['Ship_Date'], dayfirst=True)


In [18]:
# Create master session
MasterSession = sessionmaker(bind=master_engine)
master_session = MasterSession()

# Retrieve existing databases
get_db_query = 'name FROM sys.databases;'

result = master_session.query(text(get_db_query)).all()
result

[('master',),
 ('tempdb',),
 ('model',),
 ('msdb',),
 ('GithubDB',),
 ('Superstore',)]

In [38]:
print(' ---------- Table creation ----------')
name_class = class_name(input_csv_file)

# Create a declarative base
Base = declarative_base()

# Dynamically create the class with the generated name
DynamicTable = type(name_class, (Base,), {
    '__tablename__': str.replace(input_csv_file, '.csv', ''),
    'id': Column(Integer, primary_key=True, autoincrement=True),
    'row_id': Column(Integer, nullable=False),
    'order_id': Column(String(), nullable=False),
    'order_date': Column(Date, nullable=False),
    'ship_date': Column(Date, nullable=False),
    'ship_mode': Column(String(), nullable=False),
    'customer_id': Column(String(50), nullable=False),
    'customer_name': Column(String(), nullable=False),
    'segment': Column(String(50), nullable=False),
    'country': Column(String(), nullable=False),
    'city': Column(String(), nullable=False),
    'state': Column(String(), nullable=False),
    'postal_code': Column(Float, nullable=True),
    'region': Column(String(), nullable=False),
    'product_id': Column(String(50), nullable=False),
    'category': Column(String(50), nullable=False),
    'sub_category': Column(String(50), nullable=False),
    'product_name': Column(String(), nullable=False),
    'sales': Column(Float, nullable=False)
})

Base.metadata.create_all(master_engine)


 ---------- Table creation ----------


In [20]:
# Check the created tables
inspector = inspect(master_engine)

table_names = inspector.get_table_names()
table_names

['superstore_final_dataset']

In [23]:
db_df['Postal_Code'] = db_df['Postal_Code'].apply(lambda x: 'No postal code' if pd.isnull(x) else x)

/tmp/ipykernel_5178/64997817.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_df['Postal_Code'] = db_df['Postal_Code'].apply(lambda x: 'No postal code' if pd.isnull(x) else x)


In [31]:
len(db_df.columns)

18

In [45]:
db_df['Order_Date'] = pd.to_datetime(db_df['Order_Date']).dt.date

/tmp/ipykernel_5178/1362648628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_df['Order_Date'] = pd.to_datetime(db_df['Order_Date']).dt.date


In [35]:
db_df.reset_index(inplace=True, drop=True)

In [48]:
for i in range(300):
    row_id = int(db_df['Row_ID'][i])
    order_id = str(db_df['Order_ID'])[i]
    order_date = db_df['Order_Date'][i]
    ship_date = db_df['Ship_Date'][i]
    ship_mode = db_df['Ship_Mode'][i]
    customer_id = str(db_df['Customer_ID'][i])
    customer_name = str(db_df['Customer_Name'][i])
    segment = str(db_df['Segment'][i])
    country = str(db_df['Country'][i])
    city = str(db_df['City'][i])
    state = str(db_df['State'][i])
    postal_code = str(db_df['Postal_Code'][i])
    region = str(db_df['Region'][i])
    product_id = str(db_df['Product_ID'][i])
    category = str(db_df['Category'][i])
    sub_category = str(db_df['Sub_Category'][i])
    product_name = str(db_df['Product_Name'][i])
    sales = float(db_df['Sales'][i])

    new_row = {
        'row_id': row_id,
        'order_id': order_id,
        'order_date': order_date,
        'ship_date': ship_date,
        'ship_mode': ship_mode,
        'customer_id': customer_id,
        'customer_name': customer_name,
        'segment': segment,
        'country': country,
        'city': city,
        'state': state,
        'postal_code': postal_code,
        'region': region,
        'product_id': product_id,
        'category': category,
        'sub_category': sub_category,
        'product_name': product_name,
        'sales': sales
    }

    try:

        # Create an instance of the class
        new_row_instance = DynamicTable(**new_row)
        
        master_session.add(new_row_instance)
        print(i, 'worked successfully')
        master_session.commit()
    except:
        print(i, 'has an error')
        master_session.rollback()


0 worked successfully
0 has an error
1 worked successfully
2 worked successfully
3 worked successfully
4 worked successfully
5 worked successfully
6 worked successfully
7 worked successfully
8 worked successfully
9 worked successfully
10 worked successfully
11 worked successfully
12 worked successfully
13 worked successfully
14 worked successfully
15 worked successfully
16 worked successfully
17 worked successfully
17 has an error
18 worked successfully
19 worked successfully
20 worked successfully
21 worked successfully
22 worked successfully
23 worked successfully
24 worked successfully
25 worked successfully
26 worked successfully
27 worked successfully
28 worked successfully
29 worked successfully
30 worked successfully
31 worked successfully
32 worked successfully
33 worked successfully
34 worked successfully
35 worked successfully
36 worked successfully
37 worked successfully
38 worked successfully
39 worked successfully
40 worked successfully
41 worked successfully
42 worked suc

IndexError: string index out of range